# Cython

Dans cette section, nous allons voir comment **améliorer les performances** d'un code Python et comment **interfacer du code C à Python** en utilisant [**Cython**](http://cython.org/).

Cette section est inspirée du [tutorial](https://www.youtube.com/watch?v=gMvkiQ-gOW8) donné par Kurt Smith à la conférence SciPy de 2015 et de la documentation Cython.

Cython est à la fois un **langage de programmation** très proche de Python ("Cython is Python with C data types"). Il permet deux choses d'obtenir:  
* du **code compilé** C à partir de code Python.  
L'idée est de commencer avec du code Python, ajouter des informations sur les types, et obtenir du code compilé plus performant.  
* un **wrapper pour des fonctions C/C++**. 
L'idée est de commencer avec du code compilé (C/C++) et l'exposer à Python avec Cython. 

Les étapes sont les suivantes:
1. écrire un fichier cython (`.pyx`)  
2. exécuter le compiler Cython pour générer du code C  
3. exécuter un compiler C pour générer une librairie compilé
4. exécuter l'interpréteur Python et demander d'importer le module

## Comment intéragir avec Cython?

On va prendre comme exemple une fonction qui renvoie une combinaison linéaire de deux entiers. 

In [ ]:
%%writefile exo_cython/python_add.py
def pyadd(a, b):
    return 3.1415926 * a + 2.718281828 * b

### 1. compiler avec distutils

On commence par d'abord écrire un fichier `.pyx`, en gros du code Python en ajoutant des informations sur le type.

In [ ]:
%%writefile exo_cython/distutils/cython_add.pyx
def add(int a, int b):
    return 3.1415926 * a + 2.718281828 * b

In [ ]:
!cp exo_cython/python_add.py exo_cython/distutils/python_add.py

Il faut ensuite créer un fichier `setup.py` qui est une sorte de Makefile Python et qui fait appel à `distutils` et `cython`:

In [ ]:
%%writefile exo_cython/distutils/setup.py
from distutils.core import setup
from Cython.Build import cythonize

setup(name="cython_add",
      ext_modules = cythonize("cython_add.pyx")
)

On peut maintenant utiliser ce fichier pour compiler le fichier Cython avec la commande:  
`$ python setup.py buil_ext --inplace`

In [ ]:
!cd exo_cython/distutils; python setup.py build_ext --inplace

On obtient alors un fichier `cython_add.so` sous unix (ou `cython_add.pyd` sous Windows) que l'on peut utiliser en l'important dans un interpréteur Python comme n'importe quel autre module Python: `import cython_add`.

Comparons les performances avec la version Python:

In [ ]:
%%writefile exo_cython/distutils/compare_add.py
from timeit import timeit
from python_add import pyadd
from cython_add import add

print('Pure Python version: %fs' % timeit('pyadd(1, 2)', 'from python_add import pyadd'))
print('Cython version: %fs' % timeit('add(1, 2)', 'from cython_add import add'))

In [ ]:
!cd exo_cython/distutils; python compare_add.py

### 2. pyximport

pyximport est un package qui permet d'import un fichier Cython comme si c'était un module Python. Il détecte s'il y a eu des changements dans le fichier Cython, recompile au besoin (ou charge le module du cache). C'est pratique pour les cas simples.

On commence par écrire notre fichier `pyx`:

In [ ]:
%%writefile exo_cython/pyximport/cython_add.pyx
def add(int a, int b):
    return 3.1415926 * a + 2.718281828 * b

Pour utiliser directement le fichier, on utilise:
<code>
import pyximport; pyximport.install()
import cython_add
print(cython_add.add(1,2))
<code>

In [ ]:
!cp exo_cython/python_add.py exo_cython/pyximport/python_add.py

Comparons les performances avec la version Python:

In [ ]:
%%writefile exo_cython/pyximport/compare_add.py
from timeit import timeit
from python_add import pyadd
import pyximport; pyximport.install()
from cython_add import add

print('Pure Python version: %fs' % timeit('pyadd(1, 2)', 'from python_add import pyadd'))
print('Cython version: %fs' % timeit('add(1, 2)', 'from cython_add import add'))

In [ ]:
!cd exo_cython/pyximport; python compare_add.py

### 3. IPython

Il existe une commande magique IPython: 
<code>
%load_ext cythonmagic
%%cython
</code>
Comme avec `pyximport`, la compilation est faite pour nous et c'est une méthode recommandée seulement pour des cas simples.

In [ ]:
%load_ext Cython

In [ ]:
%%cython
def add(int a, int b):
    return 3.1415926 * a + 2.718281828459045 * b

In [ ]:
def pyadd(a, b):
    return 3.1415926 * a + 2.718281828459045 * b

In [ ]:
%timeit add(2, 3)

In [ ]:
%timeit pyadd(2, 3)

## Le langage Cython

### 1.1 cdef

`cdef` est la porte d'entrée de C en Python. Il permet de déclarer des variables locales, des fonctions C et des extension types.

In [ ]:
%%cython
def add(int a, int b):
    cdef:
        float c
        float d = 2.718281828459045
    c = 3.1415926  
    return c * a + d * b

In [ ]:
%timeit add(2, 3)

On peut aussir définir une fonction C:
<code>
cdef float def add(int a, int b):
    cdef:
        float c = 3.1415926, d = 2.718281828459045
    return c * a + d * b
<code>

Ou même une extension type (classe bas niveau):  
<code>
cdef class Particle(object): 
    cdef float...
</code>

D'autres declarations cdef:  

cdef      | signification  
--------- | -------------  
cdef int i, j, k | déclare des entiers C   
cdef char *s | déclare une string C  
cdef float x = 0.0 | déclare et initialise un float C  
cdef list names | déclare une liste Python statique  
cdef dict dd = {} | déclare et initialise un dictionnaire Python  

### 1.2 Comment déclarer une fonction en Cython?

Il y a trois manières de déclarer une fonction en Cython:  
* les fonctions `def`: disponibles en Python et Cython. Elles prennent et retournent un objet Python.  
* les fonctions `cdef`: fonction C. On ne peut pas y accéder depuis Python.  
* les fonctions `cpdef`: prennent le meilleur des deux mondes. Deux fonctions sont générées: une fonction C et un wrapper Python autour de cette fonction. 

In [ ]:
%%writefile exo_cython/inc1.pyx
def inc(int num, int offset):
    return num + offset

def inc_seq(seq, offset):
    result = []
    for val in seq:
        res = inc(val, offset)
        result.append(res)
    return result

In [ ]:
%%writefile exo_cython/setup1.py
from distutils.core import setup
from Cython.Build import cythonize

setup(name="inc1",
      ext_modules = cythonize("inc1.pyx")
)

In [ ]:
!cd exo_cython; python setup1.py build_ext --inplace

In [ ]:
%%writefile exo_cython/inc2.pyx
cdef int fast_inc(int num, int offset):
    return num + offset

def fast_inc_seq(seq, offset):
    result = []
    for val in seq:
        res = fast_inc(val, offset)
        result.append(res)
    return result

In [ ]:
%%writefile exo_cython/setup2.py
from distutils.core import setup
from Cython.Build import cythonize

setup(name="inc2",
      ext_modules = cythonize("inc2.pyx")
)

In [ ]:
!cd exo_cython; python setup2.py build_ext --inplace

In [ ]:
!cd exo_cython; python -c 'import inc1; print(inc1.inc(1, 3))'

In [ ]:
!cd exo_cython; python -c 'import inc1; a = range(4); print(inc1.inc_seq(a, 3))'

In [ ]:
!cd exo_cython; python -c 'import inc2; print(inc2.fast_inc(1, 3))'

In [ ]:
!cd exo_cython; python -c 'import inc2; a = range(4); print(inc2.fast_inc_seq(a, 3))'

In [ ]:
%%writefile exo_cython/inc3.pyx
cpdef int fast_inc(int num, int offset):
    return num + offset

def inc_seq(seq, offset):
    result = []
    for val in seq:
        res = fast_inc(val, offset)
        result.append(res)
    return result

In [ ]:
%%writefile exo_cython/setup3.py
from distutils.core import setup
from Cython.Build import cythonize

setup(name="inc3",
      ext_modules = cythonize("inc3.pyx")
)

In [ ]:
!cd exo_cython; python setup3.py build_ext --inplace

In [ ]:
!cd exo_cython; python -c 'import inc3; print(inc3.fast_inc(1, 3))'

In [ ]:
!cd exo_cython; python -c 'import inc3; a = range(4); print(inc3.inc_seq(a, 3))'

### 1.3 Exercice de typage

Nous allons nous entraîner à ajouter des informations sur les type.  
La fonction `hamming.py` a deux fonctions simples qui calculent la distance de Hamming entre deux strings: `hamming_sum()` et `hamming_loop()`.

In [ ]:
%%writefile exo_cython/typing/hamming.py
def hamming_sum(s0, s1):
    if len(s0) != len(s1):
        raise ValueError()
    return sum(c0 != c1 for (c0, c1) in zip(s0, s1))

def hamming_loop(s0, s1):
    if len(s0) != len(s1):
        raise ValueError()
    count = 0
    for i in range(len(s0)):
        count += (s0[i] != s1[i])
    return count

Ecrire une version Cython `exo_cython/typing/hamming_cython.pyx` en ajoutant des informations sur les types pour agmentant la performance du code. Vous pouvez utiliser `exo_cython/typing/setup_hamming.py` pour compiler votre code et `exo_cython/typing/test_hamming.py` pour comparer les performances.

## 2. Comment interfacer des fonctions C en Python?

**Remarque:** Cython n'est pas la seule méthode pour interfacer du code C ([plus de détails ici](http://www.scipy-lectures.org/advanced/interfacing_with_c/interfacing_with_c.html)), toutefois Cython est la plus avancée et celle à considérer en premier. 

On va prendre comme exemple l'interfaçage de la fonction C `sin`.

In [ ]:
%%writefile exo_cython/sin_module.pyx
# first include the header file you need
cdef extern from "math.h":
    # describe the interface for the functions used
    double sin(double x)
    
def get_sin(double x):
    # strlen can now be used from Cython code (but not python)
    return sin(x)

Cython ne lit pas le fichier C, on a donc besoin de l'informer des déclarations que l'on va utiliser.  
On doit aussi créer une `def` fonction pour que la fonction soit accessible depuis Python. 

On doit ensuite écrire un `setup`:

In [ ]:
%%writefile exo_cython/setup_sin.py
from distutils.core import setup, Extension
from Cython.Build import cythonize

s = Extension(name="sin_module",
                sources=["sin_module.pyx"])
setup(ext_modules=cythonize([s]))
# setup(name="sin_module", ext_modules=cythonize("sin_module.pyx"))

In [ ]:
!cd exo_cython; python setup_sin.py build_ext --inplace

In [ ]:
!ls exo_cython/sin*

In [ ]:
!cd exo_cython; python -c 'import sin_module; print(sin_module.sin)'

In [ ]:
!cd exo_cython; python -c 'import sin_module; print(sin_module.get_sin)'

### Exercice: 

Dans le dossier `exo_cython` se trouve la fonction `levenshtein.c` à interfacer. Pour cela, vous devez compléter `levenshtein_cython.pyx` que vous pourrez compiler avec `setup.py` et dont vous pourrez comparer les résultats avec la solution en utilisant `test_levenstein.py`. 